# Download Twitter followers for a set of users

### Notebook Author: Nikhil Utane


In [1]:
# Pip install GetOldTweets3 if you don't already have the package
# !pip install GetOldTweets3

# Imports
import GetOldTweets3 as got
import pandas as pd
import time
import tweepy
import csv
import sys
from collections import Counter 
import requests
import http.client, urllib
import re
import glob
import os

#### Read all the security tokens from a keys.py file

In [2]:
from keys import keys #keep keys in separate file, keys.py

consumer_key = keys['consumer_key']
consumer_secret = keys['consumer_secret']
access_token = keys['access_token']
access_token_secret = keys['access_token_secret']
pushover_token = keys['pushover_token']
pushover_user = keys['pushover_user']

#### I am using Pushover to notify me if any cell stops running

In [3]:
def pushoverNotify():
    conn = http.client.HTTPSConnection("api.pushover.net:443")
    conn.request("POST", "/1/messages.json",
      urllib.parse.urlencode({
        "token": pushover_token,
        "user": pushover_user,
        "message": "Cell finished execution",
      }), { "Content-type": "application/x-www-form-urlencoded" })
    r=conn.getresponse()
    print(r.status, r.reason)

In [4]:
from IPython.core.magic import register_cell_magic

# Register the magic so that later any cell that we want to be notified on exception can be done
@register_cell_magic('handle')
def handle(line, cell):
    try:
        exec(cell)        
    except Exception as e:
        pushoverNotify()
        raise # if you want the full trace-back in the notebook


### Get List of Followers. 
#### We are getting the IDs since the rate limit for that is quite high ~45000 per 15 mins vs ~3000 for usernames
#### Then we'll convert ID to username and using GetOldTweets3 to download in bulk going as far back as 2014

In [5]:
# Define all the user configuration here
side = "left"

In [6]:
followers_folder = "../data/" + side + "/followers/"
tweets_folder = "../data/" + side + "/tweets/"
handles_file = "../data/" + side + "_handles.txt"
followers_id_file = followers_folder + "all_followers_id.txt"
followers_id_dedup_file = followers_folder + "all_followers_id_dedup.txt"
followers_username_file = followers_folder + "all_followers_username.txt"
fetched_username_files = tweets_folder + "fetched_list.txt"
GetOldTweets3_bin = "/home/nikhil/packages/GetOldTweets3/bin/GetOldTweets3"
processed_path=tweets_folder + "/processed"
processed_tweets_file=processed_path + "/all_tweets.txt"
cleaned_tweets_file=processed_path + "/all_tweets_cleaned.txt"

In [11]:
# Below source code credit: https://gist.github.com/PandaWhoCodes/46f58fdead71f4c71453d9ed1e21adf8
# Credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

def get_and_save_followers(user_name):
    """
    get a list of all followers of a twitter account
    :param user_name: twitter username without '@' symbol
    :return: list of usernames without '@' symbol
    """
    followers = []
    with open(followers_folder + user_name + "_followers_id.csv", 'w',encoding="utf-8") as output:
        for page in tweepy.Cursor(api.followers_ids, screen_name=user_name, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True).pages():
            sys.stdout.write(".")
            sys.stdout.flush()
            try:
                #followers.extend(page)
                for user_id in page:
                    output.write('%s\n' % user_id)
            except tweepy.TweepError as e:
                print("Going to sleep:", e)
                # Sleeping to slow down. Else we hit rate limit often
                time.sleep(60)
    return followers

### Read the initial list of handles

In [ ]:
%%handle

with open(handles_file) as f:
    handles = [line.rstrip() for line in f]
    
for handle in handles:
    print("Getting followers for " + handle)
    followers = get_and_save_followers(handle)    
    print("Done.")
    

### Merge, de-duplicate and sort the followers list

In [7]:
print(followers_folder)
print(followers_id_file)
!echo $followers_folder/*.csv | xargs cat > $followers_id_file

../data/left/followers/
../data/left/followers/all_followers_id.txt


In [16]:
# initializing list 
with open(followers_id_file) as f:
    id_list = [line.rstrip() for line in f]

# printing original list 
print("Number of ids before dedup: %d" % len(id_list)) 

# using Counter.most_common() + list comprehension 
# sorting and removal of duplicates 
id_dedup = [key for key, value in Counter(id_list).most_common()] 

# print result 
print("Number of ids after dedup: {}. Percent reduced: {}".format(len(id_dedup), int((len(id_dedup)*100)/len(id_list))) )

with open(followers_id_dedup_file, "w") as output:
    for user_id in id_dedup:
        output.write('%s\n' % user_id)

Number of ids before dedup: 15360681
Number of ids after dedup: 8174646. Percent reduced: 53


### Convert IDs to usernames for GetOldTweets3 to fetch in bulk

In [ ]:
%%handle
# We are doing a GET on a twitter link and parsing our the username, fastest way with no rate limiting
found = not_found = last_index = 0
user_list = []

# If you are resuming from somewhere in the middle, then uncomment below lines 
# and specify the last converted ID
print("Reading ID file");
with open(followers_id_dedup_file) as f:
    id_dedup = [line.rstrip() for line in f]

last_index = id_dedup.index("1108670208607174656")
del id_dedup[0:last_index+1]

with open(followers_username_file, "w") as output:
    count = last_index + 1
    for user_id in id_dedup:
        print("[%d] Converting %s" % (count, user_id) , end=' ');
        r = requests.get('https://twitter.com/intent/user?user_id=' + user_id)
        user_search=re.search('<title>.*\(@(.*)\).*</title>', r.content.decode('utf-8'), re.IGNORECASE)
        if user_search:
            username = user_search.group(1)            
            user_list.append(username)
            output.write('%s\n' % username)
            found += 1
            print("=> %s" % username);
        else:
            not_found += 1
            print("ID %s not found" % user_id);
        count += 1
        
    print("%d usernames found. %d not found." % found, not_found)   

Reading ID file
[5239] Converting 1676066965 => _satya2013
[5240] Converting 56391919 => Dr_Aqsa_Shaikh
[5241] Converting 855320152534949888 => MUSICALRAVI
[5242] Converting 331139323 => AlamKhursheed16
[5243] Converting 943273381 => genocidalwatch
[5244] Converting 2483123348 => basha_gbd
[5245] Converting 114375529 => alafroz
[5246] Converting 3042638899 => tjbamal
[5247] Converting 363672233 => hi2deva
[5248] Converting 545827473 => CaAjmalali
[5249] Converting 2480149844 => aqshaan
[5250] Converting 1210800139 => rabindradanre
[5251] Converting 708897650 => VishnuJerome
[5252] Converting 3270224028 => RakshithPradum1
[5253] Converting 2878159046 => Shamim_asif20
[5254] Converting 2534852109 => mnkhafil
[5255] Converting 1081939701815234560 => sachasid70
[5256] Converting 1316152746 => DrishtiWahi2
[5257] Converting 120282286 => Sham_Ash
[5258] Converting 20322596 => rpsingh15
[5259] Converting 525416536 => k_saifudheen
[5260] Converting 4883298308 => Nadeem_Dilli
[5261] Converting 

[5399] Converting 1233237047873298433 => junedkhanmewati
[5400] Converting 875952621898649600 => khillare_nagesh
[5401] Converting 1233061030185570304 => Karco44926885
[5402] Converting 1211972462251872256 => ms22951
[5403] Converting 1232521069216452608 => Twiteroo8
[5404] Converting 1187066512630149120 => SayeedSarwarKh1
[5405] Converting 166450788 => WantedImram
[5406] Converting 1229459859604672512 => pandeyji_abhi
[5407] Converting 713252669792456704 => SakruddeenQadri
[5408] Converting 1232730971935559680 => ManirAl24530980
[5409] Converting 1232904038716297217 => Namenotrequire
[5410] Converting 1217729177954082817 => GausMohamad2
[5411] Converting 1025673244844601344 => HumaMirza19
[5412] Converting 1230856054604353537 => SameerBoss20
[5413] Converting 2466726272 => kabir_quraishi
[5414] Converting 1229726602071769089 => TweeticianVRG
[5415] Converting 757588764151558145 => pritamsaha758
[5416] Converting 1232728509543518208 => IAMJnehal
[5417] Converting 1217696470213656576 =>

[5550] Converting 976503418087923712 => iamfsid
[5551] Converting 1216453248107696128 ID 1216453248107696128 not found
[5552] Converting 1058377071674634240 => JeeneKaAdhikar
[5553] Converting 836585448105512960 => Azizshekh14
[5554] Converting 1226913410069123072 => HarvijaySingh13
[5555] Converting 1216388204967776262 => SatyaPr90260478
[5556] Converting 1178720980509675520 => AngryAazam
[5557] Converting 729260246502391808 => OfficeOfShadab
[5558] Converting 1226853735520927744 => abdulkh81655612
[5559] Converting 1226706233429610497 => rajukumarkann12
[5560] Converting 1226770331190190081 => JawedAk94120808
[5561] Converting 1212301802655240192 => Pradeep87864398
[5562] Converting 1226730864773664774 => Afeef_16
[5563] Converting 1223649368944889861 => MushtsqSuhail
[5564] Converting 1226615521119850497 => MohdFai66208837
[5565] Converting 1215205031453941766 => rajeevp83579110
[5566] Converting 1106614003084734464 => MazharA28330496
[5567] Converting 1222176369070596096 => Lateefk

[5702] Converting 1199370819735998464 => Shahnaw66655085
[5703] Converting 1210864613782196229 => nobadi_mr
[5704] Converting 1207724462709149697 => lier2345
[5705] Converting 344687605 => shaikhrafi007
[5706] Converting 1217013887725273088 => PrinceK82411663
[5707] Converting 4358470634 => HasnatK56034256
[5708] Converting 844237760957485057 => imparvez_21
[5709] Converting 1212644227693215744 => MDLITON45428790
[5710] Converting 811560566577721344 => Rajesh01814
[5711] Converting 1222599706385272832 => nirvair44975966
[5712] Converting 1222610521137926144 => BagwanRuman
[5713] Converting 1029419670682497025 => ZainyAli3
[5714] Converting 251931158 => iamsairabano
[5715] Converting 1216625091489890306 => manpree57639334
[5716] Converting 1222542409466175490 => india_two
[5717] Converting 1215934799187038208 => imaseemit
[5718] Converting 1222520089196384256 => Hari62269785
[5719] Converting 1222521534629085185 => RashidManan2
[5720] Converting 1222159042375602176 => Asif64829605
[5721

[5854] Converting 1219141229511938048 => RashidA84473957
[5855] Converting 1149206739197153280 => MohdAti80606034
[5856] Converting 1139954441715474433 => skmdnijamuddin2
[5857] Converting 1214589310172827649 => SayedAliAlam1
[5858] Converting 1213090459301466114 => shafiqr19280376
[5859] Converting 1181540881540640768 => Amit78166089
[5860] Converting 1218924616703344640 => MaheshriSagar
[5861] Converting 1218894550099353600 => Shaukat33923881
[5862] Converting 1218554248364883968 => AzhanMomin2
[5863] Converting 1215108410770722816 => IrshadS65853807
[5864] Converting 2204470717 => rijvanmansuri1
[5865] Converting 1218731432262041600 => SranVicki
[5866] Converting 190275188 => pavi2051
[5867] Converting 1218608655802568704 => HussainGafur
[5868] Converting 1193903764664512512 => RavindraKumR3
[5869] Converting 1218563483781844998 => Zahir34497448
[5870] Converting 1212587665381445632 => AnwalaMangera
[5871] Converting 1218545714931744768 => SanjayG62402769
[5872] Converting 121853534

...

[9052] Converting 2714371365 => mahipalverma83
[9053] Converting 873475614409318400 => Tejeswa76357425
[9054] Converting 829931950861217792 => AjayAjay3051
[9055] Converting 1228422679579021314 => MSAnsar84772229
[9056] Converting 1169923327579709440 => AzeemKh18124996
[9057] Converting 1106097053136936960 => AsimMallick5
[9058] Converting 1210513417519484928 => IAmRahim4
[9059] Converting 1179108652344598528 => THOINDIA
[9060] Converting 969450891525881856 => ShivChandraRamm
[9061] Converting 1180661455303004165 => Basharaali0353
[9062] Converting 1194315531894935552 => ayanchouder
[9063] Converting 939941510849728512 => WasiAhm85046507
[9064] Converting 1233632344755646466 => JeitendraS
[9065] Converting 1231193343276802049 => MohdNij38102621
[9066] Converting 1234760821173035009 => RahulHanotiya20
[9067] Converting 1234726505730588673 ID 1234726505730588673 not found
[9068] Converting 1234723018003402752 => Bhupend23839474
[9069] Converting 1234527971735588870 => Rafiqpadiyar6g1
[90

[9203] Converting 1232336992974696448 => realmaroof
[9204] Converting 1232282996042092551 => EkbalHu96283913
[9205] Converting 899921495593373696 => Amreenmirza959
[9206] Converting 1223657074011201536 => AbdulAzeemAsad3
[9207] Converting 1232286780352679939 => GulamAmer2
[9208] Converting 364150662 => parthaghosh7
[9209] Converting 1232272874104377349 => HODORIND
[9210] Converting 1232227867817365507 => NitikShukla5
[9211] Converting 1231873819284926465 => bMw983HrtiMxuQg
[9212] Converting 197670275 => sam_need_a_name
[9213] Converting 1232114243593527297 => ExEngineer15
[9214] Converting 946304070159032321 => Pravesh66106483
[9215] Converting 1230817371826188288 => VAdhiveshan
[9216] Converting 1231717891063468032 => DickySingh90
[9217] Converting 1175299630558433280 => msalman186
[9218] Converting 709706606028529664 => Saurabh19019145
[9219] Converting 1204643678905454592 => ReshmaV20
[9220] Converting 795340052276645889 => Muhammadaarshad
[9221] Converting 1231939960493821959 => Aa

[9357] Converting 1227131961434443777 => NOMAD_000
[9358] Converting 1073575273541615617 => MdSadda69561267
[9359] Converting 1215236143316254720 => BorioJhoka
[9360] Converting 1212084917846167552 => HabibSk11423355
[9361] Converting 1219878569213448192 => MB17363115
[9362] Converting 1226956278439849985 => aitzazahmad22
[9363] Converting 1182124397538078720 => GoldiKu20873693
[9364] Converting 1226826916918972416 => KhanMoh30116639
[9365] Converting 1226921714321223681 => reachshashwat
[9366] Converting 1226884763849375744 => MUMSHADAHMADSI1
[9367] Converting 1226904307640877057 => GBilimale
[9368] Converting 1210143085419610114 => Zulfika99657052
[9369] Converting 1226574845686628353 => MTasavour
[9370] Converting 1202277809298690048 => iahmsuhail
[9371] Converting 1226457260072697856 => RedPencil20
[9372] Converting 1216646488845115392 => Muhamma51796285
[9373] Converting 359217105 => Arahaman400
[9374] Converting 1226447857135296512 => sonubaba112
[9375] Converting 2980057525 => 1

[9512] Converting 1223099469400399872 => ilf99209545
[9513] Converting 1094886063837872128 => Abdul87511590
[9514] Converting 1222854024199729154 => Manu32047554
[9515] Converting 1214101385521725440 => ParvezRafat2
[9516] Converting 1223263421283491845 => Rajrohilla005g1
[9517] Converting 1222735494221451264 => Tapan24985160
[9518] Converting 1222076342533074944 => BravoSwara
[9519] Converting 1223237884712562688 => Naseer___13
[9520] Converting 117006308 ID 117006308 not found
[9521] Converting 1221267956635598849 => DilbagKharb2
[9522] Converting 1214802899865202690 => AniketA98721028
[9523] Converting 1147873089012690945 => pratikmankar92
[9524] Converting 893093478950813696 => JuSabine2
[9525] Converting 1072714731306274816 => RSinha25776487
[9526] Converting 1220375065990254592 => BaidyanathChat6
[9527] Converting 1208219866986995713 => AzamKha10352412
[9528] Converting 1727694937 => JavedAFarooqui
[9529] Converting 306597191 => 2007_anas
[9530] Converting 1222904915804090375 => 

[9665] Converting 1217041270570250240 => AGConsulting9
[9666] Converting 1216470409056382976 => Ataurra13006819
[9667] Converting 1220013246091911169 => SDMohid5
[9668] Converting 1181861313485520896 => PankajP42900533
[9669] Converting 2983182799 => A_Wajid_Khan
[9670] Converting 922054155627151360 => for_bats
[9671] Converting 1219957459785277441 => Mahmood05897341
[9672] Converting 1162209612449234944 => sultanakaisar
[9673] Converting 1218240020395347969 => ArnabCowsami
[9674] Converting 982508670960447488 => GulJaved23
[9675] Converting 1219884829904236545 => TheUnpopularO13
[9676] Converting 1219825500631138305 => RaviKum71383948
[9677] Converting 1206806943416901632 => ali08784264
[9678] Converting 821197076 => RIO_644
[9679] Converting 1216597866648653829 => BittuChopra7
[9680] Converting 1219826423734472704 => Ishwar211
[9681] Converting 1219792845474680832 ID 1219792845474680832 not found
[9682] Converting 1218836144655339520 => SohrabK15452194
[9683] Converting 1219255035030

### Run GetOldTweets3 to download tweets upto Jan 2014 if available

In [ ]:
# Function the pulls tweets from a specific username and turns to csv file
# Parameters: (list of twitter usernames), (max number of most recent tweets to pull from)
def username_tweets_to_csv(username, count):
    # Creation of query object
    tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
                                            .setSince("2014-01-01")\
                                            .setMaxTweets(count)\
                                            .setEmoji("unicode")
    try:
        # Creation of list that contains all tweets
        tweets = got.manager.TweetManager.getTweets(tweetCriteria)

        # Creating list of chosen tweet data
        user_tweets = [[tweet.date, tweet.text] for tweet in tweets]

        # Creation of dataframe from tweets list
        tweets_df = pd.DataFrame(user_tweets, columns = ['Datetime', 'Text'])

        # Converting dataframe to CSV
        tweets_df.to_csv(tweets_folder + '/original/{}-{}k-tweets.csv'.format(username, int(count/1000)), sep=',')
    except:
        print("Caught Rate limit Exception. Sleeping...")
        time.sleep(200)

In [ ]:
print("Reading followers")
with open(followers_username_file) as f:
    user_list = [line.rstrip() for line in f] 

print("Reading already fetched usernames")
with open(fetched_username_files) as f:
    fetched_list = [line.rstrip() for line in f]

with open(fetched_username_files, "a+") as output:
    count = 0
    for username in user_list:    
        if username not in fetched_list:
            print("[%d] Fetching tweets for %s" % (count, username))
            username_tweets_to_csv(username, 0)    
            output.write('%s\n' % username)
        else:
            print("User %s already fetched. Skipping" % username)

        count += 1

### Capture all tweets into a single file

In [7]:
# Set the path one by one and run the below two cells appropriately to generate a dataframe of tweets
path = '/home/nikhil/packages/GetOldTweets3/bin/tweets/' + side 
#path = tweets_folder + "/original"

In [9]:
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

### Run one of the below - Temporary TBD

In [10]:
leaders_tweets_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
followers_tweets_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
# Tweets download using function call of GetOldTweets3 vs using the binary have different casing for column names
# So convert below one to lowercase before merging
followers_tweets_df.columns = followers_tweets_df.columns.str.lower()
followers_tweets_df['text']

In [12]:
#frames = [leaders_tweets_df, followers_tweets_df]
frames = [leaders_tweets_df]
all_tweets_df = pd.concat(frames)

In [14]:
all_tweets_df.text.to_csv(processed_tweets_file,index=False)

### Do Text Pre-processing - Cleanup tweets

In [15]:
# Pre-processing: 
# 1) Remove URL
# 2) Keep tweets greater than 30 characters

MIN_CHARS = 30
with open(processed_tweets_file) as f:
    all_tweets = [line.rstrip() for line in f]    
    
    with open(cleaned_tweets_file, 'w',encoding="utf-8") as output:
        for tweet in all_tweets:            
            tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet, flags=re.MULTILINE)
            tweet = re.sub("([^\x00-\x7F])+"," ",tweet)
            tweet = ' '.join(tweet.split()) 
            tweet = tweet.replace('&amp;', '&')            
            if len(tweet) > MIN_CHARS:
                output.write('%s\n' % tweet)

### Shuffle, De-duplicate and Split to train + test

In [29]:
# Unable to get to work in Jupyter. Run from shell
#!awk '!seen[$0]++' all_tweets_cleaned.txt > all_tweets_dedup.txt
#!shuf all_tweets_dedup.txt > all_tweets_dedup_shuf.txt
#!mv xaa train.txt
#!mv xab test.txt

/bin/sh: 1: Syntax error: word unexpected (expecting ")")


### Download tweets using the GetOldTweets3 binary (appears to be doing it faster)

In [ ]:
# I used this to download tweets for all the 'leaders' (the original lef or right handles list)
#%%handle

since_list=['2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01']
until_list=['2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31']

tweets_bin_folder = tweets_folder + "original/leaders/"
    
with open(handles_file) as f:
    handles = [line.rstrip() for line in f]
    
for handle in handles:
    print("Getting tweets for " + handle)
    
    for since, until in zip(since_list, until_list):
        outfile = handle + "_" + since.split('-')[0] + ".csv"
        cmd = GetOldTweets3_bin + " --username " + handle + \
        " --since " + since + " --until " + until + \
        " --maxtweets 0 --emoji unicode --output " + tweets_bin_folder + outfile 
        print(cmd)
        os.system(cmd)
        print("%s Done." % outfile)
        time.sleep(60)        

### Some files are not downloaded properly. Re-download them 

In [ ]:
# Re-download those files which have size 84 bytes

all_files = glob.glob(tweets_bin_folder + "/*.csv")

for filename in all_files:
    statinfo = os.stat(filename)
    if statinfo.st_size is 84:
        filename = os.path.basename(filename)
        #print("%s is having size 84" % filename)
        handle=filename.split('_20')[0]
        year=filename.split('.csv')[0][-4:]
        #print("Re-fetching for %s & %s" % (handle, year))
        outfile = handle + "_" + year + ".csv"
        cmd = GetOldTweets3_bin + " --username " + handle + \
        " --since " + year + "-01-01" + " --until " + year + "-12-31" + \
        " --maxtweets 0 --emoji unicode --output " + tweets_bin_folder + outfile 
        print(cmd)
        os.system(cmd)
        print("%s Done." % outfile)
        time.sleep(2)

### Unused Code